In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import BertGenerationConfig

In [2]:
dataset_name="knkarthick/dialogsum"
dataset= load_dataset(dataset_name)

In [3]:
example_indices= [40,200]

dash_line = '_'.join('' for x in range(100))
for i,index in enumerate(example_indices):
    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('baseline human summary:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
baseline human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________

___________________________________________________________________________________________________
Exam

In [4]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
tokenizer= AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [6]:
sentence="hey there bitch"

sentence_encoded= tokenizer(sentence,return_tensors='pt')

sentence_decoded = tokenizer.decode(
    sentence_encoded['input_ids'][0],
    skip_special_tokens=True
)

print('ENCODED SENTENCE:')
print(sentence_encoded['input_ids'][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([    3, 13133,   132,   720,   524,     1])

DECODED SENTENCE:
hey there bitch


In [7]:
for i,index in enumerate(example_indices):
    dialogue= dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    input= tokenizer(dialogue,return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            input['input_ids'],
            max_new_tokens =50
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('example',i+1)
    print(dash_line)
    print(f'input prompt:\n{dialogue}')
    print(dash_line)
    print(f'Baseline human summary:\n{summary}')
    print(dash_line)
    print(f' model generation-without prompt:\n{output}\n')

___________________________________________________________________________________________________
example 1
___________________________________________________________________________________________________
input prompt:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
Baseline human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
 model generation-without prompt:
Person1: It's ten to nine.

_____________________________________________

ZERO SHOT INFERENCE

In [8]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
summarize the following conversation.
{dialogue}

summary:
"""
    input = tokenizer(prompt,return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            input['input_ids'],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('example',i+1)
    print(dash_line)
    print(f'input prompt:\n{prompt}')
    print(dash_line)
    print(f'Baseline human summary:\n{summary}')
    print(dash_line)
    print(f' model generation- zero shot:\n{output}\n')
    

___________________________________________________________________________________________________
example 1
___________________________________________________________________________________________________
input prompt:

summarize the following conversation.
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

summary:

___________________________________________________________________________________________________
Baseline human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
 model generation- zero shot:
The train is about to leave

ONE SHOT

In [9]:
def make_prompt(example_indices_full,example_index_to_summarize):
    prompt= ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        prompt += f"""  
Dialogue:
{dialogue}

summary:
{summary}
"""  
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    prompt += f"""  
dialogue:
{dialogue}

summary:
"""   
    return prompt
    



In [10]:
example_indices_full  = [40]
example_index_to_summarize = 200
one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(one_shot_prompt)

  
Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
  
dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more 

In [11]:
summary = dataset['test'][example_index_to_summarize]['summary']

input = tokenizer(one_shot_prompt,return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        input['input_ids'],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens=True
)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


FEW SHOT INFERENCE

In [12]:
example_indices_full=[40,80,120,160]
example_index_to_summarize=200

few_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(few_shot_prompt)

  
Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
  
Dialogue:
#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. May, can y

In [13]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt,return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        input['input_ids'],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL_GENERATION:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (927 > 512). Running this sequence through the model will result in indexing errors


___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
MODEL_GENERATION:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.
